In [1]:
import pandas as pd

In [2]:
df = pd.read_excel("input.xlsx")

In [3]:
df.columns

Index(['Amount of catalyst (g) ', 'Hole scavenger (ml)',
       'Light Intensity of the photoreactor (W)', 'Time (h)',
       'TOF (µmol g-1 h-1)', 'AQY %'],
      dtype='str')

# Option 1 — NSGA-II Multi-Objective Optimization 

Uses:

RandomForest surrogate model

NSGA-II evolutionary optimizer (pymoo)

In [4]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from pymoo.core.problem import Problem
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.optimize import minimize

pd.options.display.float_format = '{:.6f}'.format

# -------------------------
# Prepare Data
# -------------------------
X = df.iloc[:, 0:4].values
y_tof = df['TOF (µmol g-1 h-1)'].values
y_aqy = df['AQY %'].values

# Train surrogate models
model_tof = RandomForestRegressor()
model_aqy = RandomForestRegressor()

model_tof.fit(X, y_tof)
model_aqy.fit(X, y_aqy)

# -------------------------
# Define Optimization Problem
# -------------------------
class PhotocatProblem(Problem):

    def __init__(self):
        xl = X.min(axis=0)
        xu = X.max(axis=0)

        super().__init__(n_var=4,
                         n_obj=2,
                         n_constr=0,
                         xl=xl,
                         xu=xu)

    def _evaluate(self, x, out, *args, **kwargs):
        tof_pred = model_tof.predict(x)
        aqy_pred = model_aqy.predict(x)

        # minimize negative = maximize
        out["F"] = np.column_stack([-tof_pred, -aqy_pred])

# -------------------------
# Run NSGA-II
# -------------------------
problem = PhotocatProblem()

algorithm = NSGA2(pop_size=80)

result = minimize(problem,
                  algorithm,
                  ('n_gen', 100),
                  seed=1,
                  verbose=True)

# -------------------------
# Results
# -------------------------
pareto_X = result.X
pareto_F = -result.F   # convert back to positive

#print("Optimal conditions:")
#print(pareto_X)

#print("Predicted TOF & AQY:")
#print(pareto_F)

# combine pareto_X and pareto_F into a DataFrame for better visualization

results_df = pd.DataFrame(pareto_X, columns=df.columns[0:4])
results_df['TOF (µmol g-1 h-1)'] = pareto_F[:, 0]
results_df['AQY %'] = pareto_F[:, 1]
print("\nPareto-optimal solutions:")
print(results_df)
#save results to Excel
results_df.to_excel("option_1_pareto_optimal_solutions.xlsx", index=False)

n_gen  |  n_eval  | n_nds  |      eps      |   indicator  
     1 |       80 |      5 |             - |             -
     2 |      160 |      7 |  0.0231233695 |         ideal
     3 |      240 |      8 |  0.1326234105 |         ideal
     4 |      320 |     11 |  0.2199340857 |         ideal
     5 |      400 |     18 |  0.1522651533 |         ideal
     6 |      480 |     28 |  0.0125603087 |             f
     7 |      560 |     42 |  0.0100263517 |         ideal
     8 |      640 |     74 |  0.0506174373 |         ideal
     9 |      720 |     80 |  0.0003049441 |             f
    10 |      800 |     80 |  0.0029729497 |             f
    11 |      880 |     80 |  0.0004342352 |             f
    12 |      960 |     80 |  0.0005789802 |             f
    13 |     1040 |     80 |  0.0018437178 |             f
    14 |     1120 |     80 |  0.0062611056 |             f
    15 |     1200 |     80 |  0.000000E+00 |             f
    16 |     1280 |     80 |  0.0000156447 |            

# Option 2 — Weighted Objective Method 

Convert multi-objective → single score.

Good when you want one best point.

In [5]:
from scipy.optimize import differential_evolution

# normalize targets
tof_norm = (y_tof - y_tof.min())/(y_tof.max()-y_tof.min())
aqy_norm = (y_aqy - y_aqy.min())/(y_aqy.max()-y_aqy.min())

model_tof.fit(X, tof_norm)
model_aqy.fit(X, aqy_norm)

def objective(x):
    x = np.array(x).reshape(1,-1)
    score = 0.5*model_tof.predict(x) + 0.5*model_aqy.predict(x)
    return -score   # maximize

bounds = list(zip(X.min(axis=0), X.max(axis=0)))

result = differential_evolution(objective, bounds)

print("Best settings:", result.x)

# save best settings to Excel
best_df = pd.DataFrame([result.x], columns=df.columns[0:4])
best_df['Predicted TOF (normalized)'] = model_tof.predict(result.x.reshape(1,-1))[0]
best_df['Predicted AQY (normalized)'] = model_aqy.predict(result.x.reshape(1,-1))[0]
best_df.to_excel("option_2_best_solution.xlsx", index=False)


Best settings: [1.34199413e-02 3.36070286e-01 2.06382115e+01 7.07853721e+00]


# Option 3 — Bayesian Optimization 

Best if dataset is small.

In [6]:
import optuna

def objective(trial):

    x = [
        trial.suggest_float("cat", X[:,0].min(), X[:,0].max()),
        trial.suggest_float("scav", X[:,1].min(), X[:,1].max()),
        trial.suggest_float("light", X[:,2].min(), X[:,2].max()),
        trial.suggest_float("time", X[:,3].min(), X[:,3].max()),
    ]

    x = np.array(x).reshape(1,-1)

    tof = model_tof.predict(x)[0]
    aqy = model_aqy.predict(x)[0]

    return tof, aqy

study = optuna.create_study(directions=["maximize","maximize"])
study.optimize(objective, n_trials=200)

print(study.best_trials)

# Extract Pareto-optimal solutions and their predicted TOF & AQY and save to Excel
pareto_solutions = []
for trial in study.best_trials:
    x = [trial.params["cat"], trial.params["scav"], trial.params["light"], trial.params["time"]]
    x = np.array(x).reshape(1,-1)
    tof = model_tof.predict(x)[0]
    aqy = model_aqy.predict(x)[0]
    pareto_solutions.append({
        "cat": x[0][0],
        "scav": x[0][1],
        "light": x[0][2],
        "time": x[0][3],
        "TOF": tof,
        "AQY": aqy
    })

pareto_df = pd.DataFrame(pareto_solutions)
pareto_df.to_excel("option_3_pareto_optimal_solutions.xlsx", index=False)

/workspaces/Photocatalytic-Parameter-Optimization/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2026-02-09 18:22:47,151] A new study created in memory with name: no-name-99962993-4823-4916-ac4f-8cd27e0f0dbb
[I 2026-02-09 18:22:47,166] Trial 0 finished with values: [0.7111114383344136, 0.5333743226660523] and parameters: {'cat': 0.01727786399356229, 'scav': 0.18695433657368127, 'light': 46.67177312421447, 'time': 1.9985524230080545}.
[I 2026-02-09 18:22:47,180] Trial 1 finished with values: [0.7745989590291478, 0.4719274934845437] and parameters: {'cat': 0.009916527158643811, 'scav': 0.28601365328623946, 'light': 39.502409095884026, 'time': 7.928159334936321}.
[I 2026-02-09 18:22:47,191] Trial 2 finished with values: [0.3720538334314634, 0.3251814264332211] and parameters: {'cat': 0.0088010462

[I 2026-02-09 18:22:47,276] Trial 8 finished with values: [0.827058317988645, 0.9337101564869746] and parameters: {'cat': 0.012992509489041675, 'scav': 0.3700671901569274, 'light': 21.05137391341302, 'time': 6.785821620262432}.
[I 2026-02-09 18:22:47,293] Trial 9 finished with values: [0.29542816143816203, 0.33737362040936825] and parameters: {'cat': 0.014779294693921242, 'scav': 0.07615758651775184, 'light': 19.32191788222296, 'time': 6.8544410457999945}.
[I 2026-02-09 18:22:47,306] Trial 10 finished with values: [0.8433732952709785, 0.6514217949121703] and parameters: {'cat': 0.016407091062032975, 'scav': 0.30920287673908187, 'light': 34.476683525059144, 'time': 6.019942716900148}.
[I 2026-02-09 18:22:47,320] Trial 11 finished with values: [0.5950327384685441, 0.3677520938890709] and parameters: {'cat': 0.008717798108521208, 'scav': 0.3815293051243448, 'light': 40.41756578651768, 'time': 10.697118079231688}.
[I 2026-02-09 18:22:47,335] Trial 12 finished with values: [0.66991315212049

[FrozenTrial(number=8, state=<TrialState.COMPLETE: 1>, values=[0.827058317988645, 0.9337101564869746], datetime_start=datetime.datetime(2026, 2, 9, 18, 22, 47, 257709), datetime_complete=datetime.datetime(2026, 2, 9, 18, 22, 47, 275997), params={'cat': 0.012992509489041675, 'scav': 0.3700671901569274, 'light': 21.05137391341302, 'time': 6.785821620262432}, user_attrs={}, system_attrs={'NSGAIISampler:generation': 0}, intermediate_values={}, distributions={'cat': FloatDistribution(high=0.02, log=False, low=0.005, step=None), 'scav': FloatDistribution(high=0.4, log=False, low=0.05, step=None), 'light': FloatDistribution(high=50.0, log=False, low=5.0, step=None), 'time': FloatDistribution(high=11.0, log=False, low=1.0, step=None)}, trial_id=8, value=None), FrozenTrial(number=10, state=<TrialState.COMPLETE: 1>, values=[0.8433732952709785, 0.6514217949121703], datetime_start=datetime.datetime(2026, 2, 9, 18, 22, 47, 293781), datetime_complete=datetime.datetime(2026, 2, 9, 18, 22, 47, 306933)

# Option 4 — Grid + Pareto Filtering 

Good for small ranges.

In [7]:
from itertools import product

grid = [
    np.linspace(X[:,i].min(), X[:,i].max(), 20)
    for i in range(4)
]

candidates = np.array(list(product(*grid)))

tof_pred = model_tof.predict(candidates)
aqy_pred = model_aqy.predict(candidates)

results = pd.DataFrame(candidates, columns=df.columns[:4])
results["tof"] = tof_pred
results["aqy"] = aqy_pred

# Pareto filter
pareto = results[
    (results["tof"] > results["tof"].quantile(0.8)) &
    (results["aqy"] > results["aqy"].quantile(0.8))
]

print(pareto)

# combine pareto_X and pareto_F into a DataFrame for better visualization and save to Excel
pareto_X = pareto.iloc[:, 0:4].values
pareto_F = pareto.iloc[:, 4:6].values

pareto_df = pd.DataFrame(np.hstack([pareto_X, pareto_F]), columns=list(df.columns[:4]) + ["TOF", "AQY"])
pareto_df.to_excel("option_4_pareto_optimal_solutions.xlsx", index=False)

        Amount of catalyst (g)   Hole scavenger (ml)  \
75750                  0.012105             0.215789   
75751                  0.012105             0.215789   
75752                  0.012105             0.215789   
75753                  0.012105             0.215789   
75770                  0.012105             0.215789   
...                         ...                  ...   
159893                 0.020000             0.400000   
159894                 0.020000             0.400000   
159895                 0.020000             0.400000   
159896                 0.020000             0.400000   
159897                 0.020000             0.400000   

        Light Intensity of the photoreactor (W)  Time (h)      tof      aqy  
75750                                 21.578947  6.263158 0.730791 0.748156  
75751                                 21.578947  6.789474 0.730791 0.748156  
75752                                 21.578947  7.315789 0.730791 0.748156  
75753          